In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [77]:
df = pd.read_csv('BakuApartmentData.csv')

In [78]:
df.head(20)

,Unnamed: 0,price,location,rooms,square,floor,new_building,has_repair,has_bill_of_sale,has_mortgage
0,0,284000,Azadlıq Prospekti m.,3,140.0,5/12,1,1,1,1
1,1,355000,Şah İsmayıl Xətai m.,3,135.0,19/20,1,1,1,1
2,2,755000,Səbail r.,4,210.0,7/18,1,1,1,1
3,3,245000,Elmlər Akademiyası m.,3,86.0,8/10,1,1,1,1
4,4,350000,Elmlər Akademiyası m.,4,174.0,12/15,1,1,1,1
5,5,255000,Nizami m.,2,93.0,10/16,1,1,1,1
6,6,410000,Ağ şəhər q.,3,133.0,4/6,1,1,1,1
7,7,235000,İnşaatçılar m.,3,130.0,9/18,1,1,1,1
8,8,125000,Qara Qarayev m.,2,63.0,15/16,1,1,1,1
9,9,207000,Həzi Aslanov m.,3,123.0,10/17,1,1,1,1


In [79]:
X = df.drop(columns = "price")
y = df["price"]

In [80]:
df.drop(columns = "Unnamed: 0")

,price,location,rooms,square,floor,new_building,has_repair,has_bill_of_sale,has_mortgage
0,284000,Azadlıq Prospekti m.,3,140.0,5/12,1,1,1,1
1,355000,Şah İsmayıl Xətai m.,3,135.0,19/20,1,1,1,1
2,755000,Səbail r.,4,210.0,7/18,1,1,1,1
3,245000,Elmlər Akademiyası m.,3,86.0,8/10,1,1,1,1
4,350000,Elmlər Akademiyası m.,4,174.0,12/15,1,1,1,1
...,...,...,...,...,...,...,...,...,...
39297,85500,9-cu mikrorayon q.,2,45.0,9/9,0,0,1,0
39298,102500,Memar Əcəmi m.,2,48.0,3/5,0,0,1,0
39299,143500,İnşaatçılar m.,3,65.0,3/5,0,0,1,0
39300,172000,Elmlər Akademiyası m.,4,90.0,3/5,0,0,1,0


In [81]:
df = df.drop(columns=["floor"], errors="ignore")

In [82]:
num_cols = ["rooms", "square", "new_building", "has_repair", "has_bill_of_sale", "has_mortgage"]
cat_cols = ["location"]


In [83]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [84]:
num_pipe = Pipeline([
    ('imput', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

In [85]:
cat_pipe = Pipeline([
    ('imput', SimpleImputer(strategy = 'most_frequent')),
    ('hot', OneHotEncoder(handle_unknown = 'ignore'))
]
)

In [86]:
preprocess = ColumnTransformer([
    ("num", num_pipe , num_cols),
    ("cat", cat_pipe , cat_cols)
])

In [87]:
model = Pipeline([
    ('pre' , preprocess),
   ('reg', LinearRegression())
])

In [88]:
model.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('pre', ...), ('reg', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains spar

In [89]:
y_pred = model.predict(X_test)

In [90]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 45001.71860987064
RMSE: 97595.6432899608
R2: 0.7514594722099532


In [93]:
df["price"].head(30)

0     284000
1     355000
2     755000
3     245000
4     350000
5     255000
6     410000
7     235000
8     125000
9     207000
10    355000
11    180000
12    210000
13    222000
14    125000
15    360000
16    270000
17    139900
18    125000
19    229000
20    229000
21    310000
22    148000
23    175000
24    210000
25    245000
26    430000
27    110000
28    175000
29    319000
Name: price, dtype: int64

In [101]:
import pandas as pd

new_home = {
    "rooms": 2,
    "square": 110,
    "location": "Şah İsmayıl Xətai m.",          # səndə necə yazılıbsa elə
    "has_repair": 1,
    "new_building": 1,
    "has_bill_of_sale": 1,
    "has_mortgage": 0
}

new_df = pd.DataFrame([new_home])

pred_price = model.predict(new_df)[0]
print("Predicted price:", pred_price)

Predicted price: 297576.62227421976


In [98]:
df["location"].value_counts()

location
İnşaatçılar m.          2834
Nəriman Nərimanov m.    2633
Nəsimi r.               2254
Şah İsmayıl Xətai m.    2171
Həzi Aslanov m.         2113
                        ... 
Savalan q.                 1
Hökməli q.                 1
Qala q.                    1
1-ci mikrorayon q.         1
Şimal DRES q.              1
Name: count, Length: 111, dtype: int64

In [ ]:
import pandas as pd

new_home = {
    "rooms": 2,
    "square": 42,
    "location": "Yasamal",          # və ya datasetdə necə yazılıbsa elə (məs: "İnşaatçılar")
    "new_building": 0,              # Xruşşovka -> köhnə tikili
    "has_bill_of_sale": 1,          # kupça var
    "has_repair": 1,                # “yaşayış üçün hazır” deyir -> çox vaxt 1 götürürlər (amma tam dəqiq deyil)
    "has_mortgage": 0               # elan demirsə 0/default seçmək olar
}

new_df = pd.DataFrame([new_home])

pred_price = model.predict(new_df)[0]
print("Predicted price:", pred_price)

ValueError: columns are missing: {'has_repair', 'new_building', 'has_bill_of_sale', 'has_mortgage'}

In [102]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)

print("Train MAE:", mean_absolute_error(y_train, y_pred_train))
print("Test  MAE:", mean_absolute_error(y_test,  y_pred_test))

print("Train R2:", r2_score(y_train, y_pred_train))
print("Test  R2:", r2_score(y_test,  y_pred_test))

Train MAE: 44385.33342127263
Test  MAE: 45001.71860987064
Train R2: 0.6902620802060366
Test  R2: 0.7514594722099532


In [103]:
import numpy as np

residuals = y_test - y_pred  # y_pred əvvəl hesablanıbsa
print("Residual mean:", np.mean(residuals))
print("Residual median:", np.median(residuals))
print("Residual min/max:", residuals.min(), residuals.max())

Residual mean: -701.2483902041973
Residual median: -562.3872514389222
Residual min/max: -1139878.8317025742 3107587.861122523


In [111]:
import pandas as pd

# 1) X-in bütün sütunlarını 0-la doldur (default 0)
new_df = pd.DataFrame([{col: 0 for col in X.columns}])

# 2) Şəkildən bildiyimiz dəyərləri yaz
new_df.loc[0, "location"] = "Masazır q."   # datasetdə necə yazılıbsa elə
new_df.loc[0, "rooms"] = 5
new_df.loc[0, "square"] = 185

# 3) Qiymət proqnozu
pred_price = model.predict(new_df)[0]
print("Predicted price:", pred_price)

Predicted price: 287245.58130796556


/var/folders/q0/y42t8q_d3wn46w1whd8krl0w0000gn/T/ipykernel_1818/3521803290.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Masazır q.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.loc[0, "location"] = "Masazır q."   # datasetdə necə yazılıbsa elə
